In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import requests
import pandas as pd

dataset_url = "https://drive.google.com/uc?id=1slcvP4WTNOefZiaqf9z_EqveFSewJjsV&export=download"

response = requests.get(dataset_url)
content = response.content

with open("dataset.csv", "wb") as file:
    file.write(content)

df = pd.read_csv("dataset.csv")



In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [6]:
df.shape

(50000, 2)

In [7]:
#Count the no. of positive and negative review
df['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [8]:
!pip install fastapi uvicorn
!pip install pyngrok
!pip install fastapi nest-asyncio pyngrok uvicorn
!pip install --upgrade fastapi
!pip install typing_extensions
!pip install --upgrade typing_extensions

!pip install fastapi uvicorn pyngrok tensorflow_hub

  Using cached fastapi-0.109.0-py3-none-any.whl (92 kB)
  Using cached starlette-0.35.1-py3-none-any.whl (71 kB)
  Attempting uninstall: starlette
    Found existing installation: starlette 0.14.2
    Uninstalling starlette-0.14.2:
      Successfully uninstalled starlette-0.14.2
  Attempting uninstall: fastapi
    Found existing installation: fastapi 0.67.0
    Uninstalling fastapi-0.67.0:
      Successfully uninstalled fastapi-0.67.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.


In [9]:
!pip show typing_extensions

Name: typing_extensions
Version: 4.9.0
Summary: Backported and Experimental Type Hints for Python 3.8+
Home-page: 
Author: 
Author-email: "Guido van Rossum, Jukka Lehtosalo, Łukasz Langa, Michael Lee" <levkivskyi@gmail.com>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: arviz, chex, fastapi, flax, google-generativeai, huggingface-hub, ibis-framework, inflect, librosa, orbax-checkpoint, panel, pydantic, pymc, pytensor, python-utils, qudida, SQLAlchemy, tensorflow, tensorflow-probability, torch, typer, uvicorn


In [10]:
!pip install fastapi==0.67.0

import tensorflow_hub as hub
import nest_asyncio
import numpy as np
from typing import List
from pyngrok import ngrok

from fastapi import FastAPI
app=FastAPI()

  Using cached fastapi-0.67.0-py3-none-any.whl (51 kB)
  Using cached starlette-0.14.2-py3-none-any.whl (60 kB)
  Attempting uninstall: starlette
    Found existing installation: starlette 0.35.1
    Uninstalling starlette-0.35.1:
      Successfully uninstalled starlette-0.35.1
  Attempting uninstall: fastapi
    Found existing installation: fastapi 0.109.0
    Uninstalling fastapi-0.109.0:
      Successfully uninstalled fastapi-0.109.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.


In [11]:
import pickle



#Load the Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

 # Define the batch size
batch_size = 1000

#Tokenize and encode the reviews in batches
x_batches = []
num_samples = len(df['review'])
num_batches = num_samples // batch_size

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = (i + 1) * batch_size
    reviews_batch = df['review'].iloc[start_idx:end_idx].tolist()
    embeddings_batch = embed(reviews_batch).numpy()
    x_batches.append(embeddings_batch)

#Process the remaining samples (if any) that don't fit in a full batch
remaining_samples = num_samples % batch_size
if remaining_samples > 0:
    start_idx = num_batches * batch_size
    reviews_batch = df['review'].iloc[start_idx:].tolist()
    embeddings_batch = embed(reviews_batch).numpy()
    x_batches.append(embeddings_batch)

#Concatenate the batches into a single numpy array
x = np.concatenate(x_batches, axis=0)

#Changing the sentiment values as 1 and 0 for positive and negative respectively
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y=le.fit_transform(df['sentiment'])

#Splitting the data into the training and the testing set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train,y_test=train_test_split(x,y,test_size=0.2, random_state=42)

#Building and training the SVM model
from sklearn.svm import SVC
model=SVC()
model.fit(x_train,y_train)

#Make the predictions
y_pred=model.predict(x_test)

#model evaluation
from sklearn.metrics import accuracy_score,confusion_matrix, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print("Accuracy:",accuracy*100, "%")
print("Confusion Matrix:")
print(cm)
print("Precison:", precision)
print("Recall:", recall)
print("F1-score:", f1)



# Save the model and label encoder using pickle
model_path = "/content/trained_model.pickle"
encoder_path = "/content/label_encoder.pickle"

with open(model_path, "wb") as file:
    pickle.dump(model, file)

with open(encoder_path, "wb") as file:
    pickle.dump(le, file)





Accuracy: 87.86 %
Confusion Matrix:
[[4361  600]
 [ 614 4425]]
Precison: 0.8805970149253731
Recall: 0.8781504266719588
F1-score: 0.8793720190779015


In [12]:
import os

model_path = "/content/trained_model.pickle"
encoder_path = "/content/label_encoder.pickle"

# Check if the model file exists
if os.path.exists(model_path):
    print("Model file exists")
else:
    print("Model file does not exist")

# Check if the label encoder file exists
if os.path.exists(encoder_path):
    print("Label encoder file exists")
else:
    print("Label encoder file does not exist")


Model file exists
Label encoder file exists


In [ ]:
import pickle
import random
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import uvicorn

import os
import nest_asyncio
from pyngrok import ngrok
import uvicorn


# specify a port
port = 8000
ngrok_tunnel = ngrok.connect(port)

nest_asyncio.apply()

# where we can visit our fastAPI app
print('Public URL:', ngrok_tunnel.public_url)

app=FastAPI()
# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["GET", "POST", "OPTIONS"],
    allow_headers=["Content-Type"],
)



class PredictRequest(BaseModel):
    review: str


@app.post("/predict")
async def predict_sentiment(request: PredictRequest):
    # Load the trained model and label encoder
    model_path = "/content/trained_model.pickle"
    encoder_path = "/content/label_encoder.pickle"

    with open(model_path, "rb") as file:
        model = pickle.load(file)

    with open(encoder_path, "rb") as file:
        label_encoder = pickle.load(file)

    # Load the Universal Sentence Encoder
    embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

    # Encode the input review
    embedding = embed([request.review]).numpy()

    # Make prediction using the trained model
    prediction = model.predict(embedding)[0]

    # Convert the numerical prediction back to a sentiment label
    sentiment_label = label_encoder.inverse_transform([prediction])[0]

    # Return the predicted sentiment label as a response
    return {"prediction": sentiment_label}

if __name__ == "__main__":
   uvicorn.run(app, host="0.0.0.0", port = 8000)

INFO:     Started server process [10927]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: https://2f37-34-125-110-178.ngrok-free.app
INFO:     210.212.2.133:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     210.212.2.133:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     210.212.2.133:0 - "POST /predict HTTP/1.1" 200 OK
